In [18]:
!pip install numpy
!pip install scikit-learn
!pip install scikit-learn pandas matplotlib
!pip install xgboost
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [19]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [20]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

In [21]:
# Define the model hyperparameters
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 8888,
}

# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

# Predict on the test set
y_pred = lr.predict(X_test)

report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.95      0.97      0.96       270
           1       0.62      0.50      0.56        30

    accuracy                           0.92       300
   macro avg       0.79      0.73      0.76       300
weighted avg       0.91      0.92      0.92       300



In [22]:
report_dict = classification_report(y_test, y_pred, output_dict=True)
report_dict

{'0': {'precision': 0.9456521739130435,
  'recall': 0.9666666666666667,
  'f1-score': 0.9560439560439561,
  'support': 270.0},
 '1': {'precision': 0.625,
  'recall': 0.5,
  'f1-score': 0.5555555555555556,
  'support': 30.0},
 'accuracy': 0.92,
 'macro avg': {'precision': 0.7853260869565217,
  'recall': 0.7333333333333334,
  'f1-score': 0.7557997557997558,
  'support': 300.0},
 'weighted avg': {'precision': 0.9135869565217392,
  'recall': 0.92,
  'f1-score': 0.9159951159951161,
  'support': 300.0}}

In [23]:
import mlflow

# Connect to local MLflow UI
mlflow.set_tracking_uri("http://127.0.0.1:5000/")
mlflow.set_experiment("First Experiment")

# Dummy params and metrics
params = {
    "penalty": "l2",
    "solver": "liblinear",
    "max_iter": 100
}

metrics = {
    "accuracy": 0.92,
    "recall_class_0": 0.88,
    "recall_class_1": 0.95,
    "f1_score_macro": 0.91
}

# Start MLflow run and log data
with mlflow.start_run():
    mlflow.log_params(params)
    mlflow.log_metrics(metrics)
    print("🚀 Logged dummy run to MLflow!")


🚀 Logged dummy run to MLflow!
🏃 View run thundering-zebra-531 at: http://127.0.0.1:5000/#/experiments/225657383368768906/runs/19c59cd33138420c9e49759612cc6446
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/225657383368768906


In [ ]:
#alternate way

import mlflow
import mlflow.sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# Load some basic data
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Train a model
lr = LogisticRegression(penalty="l2", solver="liblinear", max_iter=100)
lr.fit(X_train, y_train)

# Set experiment and tracking
mlflow.set_tracking_uri("http://127.0.0.1:5000/")
mlflow.set_experiment("First Experiment")

params = {
    "penalty": "l2",
    "solver": "liblinear",
    "max_iter": 100
}

metrics = {
    "accuracy": lr.score(X_test, y_test),
    "recall_class_0": 0.88,
    "recall_class_1": 0.95,
    "f1_score_macro": 0.91
}

# Start MLflow run
with mlflow.start_run():
    mlflow.log_params(params)
    mlflow.log_metrics(metrics)
    
    # ✅ Log the trained model
    mlflow.sklearn.log_model(lr, artifact_path="sklearn-model")
    
    print("🚀 Logged model and metrics to MLflow")


2025/04/09 15:23:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🚀 Logged model and metrics to MLflow
🏃 View run flawless-midge-296 at: http://127.0.0.1:5000/#/experiments/225657383368768906/runs/daba483e10094827a817834c75f8a4b6
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/225657383368768906


In [ ]:
# Alternate way

import mlflow
import mlflow.sklearn
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from mlflow.models.signature import infer_signature
import pandas as pd

# Load data
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Train model
lr = LogisticRegression()
lr.fit(X_train, y_train)

# Inferred signature
signature = infer_signature(X_train, lr.predict(X_train))
input_example = pd.DataFrame(X_train[:2], columns=[f"feature_{i}" for i in range(X.shape[1])])

mlflow.set_tracking_uri("http://127.0.0.1:5000/")
mlflow.set_experiment("First Experiment")

with mlflow.start_run():
    mlflow.log_param("model_type", "LogisticRegression")
    mlflow.log_metric("accuracy", lr.score(X_test, y_test))
    
    # ✅ Logging with input_example and signature
    mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="sklearn-model",
        input_example=input_example,
        signature=signature
    )


🏃 View run judicious-dove-133 at: http://127.0.0.1:5000/#/experiments/225657383368768906/runs/de2497bc37b74803862c87c48b271546
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/225657383368768906
